In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [4]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/DRM-PMSHELTER/DRM-PMSHELTER.csv", encoding='cp949')
print("원본 크기:", df.shape)
display(df.head())

# 2. 구 컬럼 생성
df = df.rename(columns={"행정동 이름": "gu"})

# 3. 수치형 변환 (이용가능인원)
df["이용가능인원"] = pd.to_numeric(df["이용가능인원"], errors="coerce").fillna(0)

# 4. 집계 (구별)
agg = df.groupby("gu").agg(
    pmshelter_count=("시설 이름", "count"),
    pmshelter_total_capacity=("이용가능인원", "sum")
).reset_index()

display(agg.head())

# 5. 점수화
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["pmshelter_count","pmshelter_total_capacity"]])

score_df = agg[["gu"]].copy()
score_df["pmshelter_count_score"] = scaled[:, 0]
score_df["pmshelter_total_capacity_score"] = scaled[:, 1]

# 6. 종합 점수 (가중 평균)
score_df["pmshelter_mean_score"] = (
    score_df["pmshelter_count_score"] * 0.4 +
    score_df["pmshelter_total_capacity_score"] * 0.6
)

# 7. 저장
score_df.to_csv("../../result/DRM/DRM-PMSHELTER_result.csv", index=False, encoding="utf-8-sig")

score_df.head()

원본 크기: (206, 13)


,순번,행정동 이름,시설 이름,주소,시설유형,이용가능인원,평일이용시간,주말이용시간,회원제 여부,X좌표,Y좌표,면적,비고
0,3,종로구,혜명경로당,서울특별시 종로구 혜화로9길 24-9,노인시설,100,09:00~18:00,NaN,NaN,199848.342439,554277.074165,NaN,NaN
1,4,중구,광희경로당,서울특별시 중구 퇴계로 303-9 광희동주민센터 4층,복지회관,30,09:00~18:00,NaN,NaN,200445.672843,551660.313597,NaN,NaN
2,5,중구,약수노인종합복지관,서울특별시 중구 다산로6길 11,복지회관,150,09:00~18:00,NaN,NaN,200823.359322,550327.417073,NaN,NaN
3,6,중구,청구e편한세상경로당,서울특별시 중구 청구로 64 청구 e편한세상 109동,노인시설,40,09:00~18:00,NaN,NaN,201490.134182,551230.446120,NaN,NaN
4,7,중구,현대아파트경로당,서울특별시 중구 다산로 36길 109 현대아파트 정문 앞,노인시설,40,09:00~18:00,NaN,NaN,202029.533020,551219.584585,NaN,NaN


,gu,pmshelter_count,pmshelter_total_capacity
0,강남구,32,320
1,강동구,4,168
2,강북구,4,160
3,강서구,20,430
4,관악구,3,460


,gu,pmshelter_count_score,pmshelter_total_capacity_score,pmshelter_mean_score
0,강남구,100.000000,57.943925,74.766355
1,강동구,9.677419,29.532710,21.590594
2,강북구,9.677419,28.037383,20.693398
3,강서구,61.290323,78.504673,71.618933
4,관악구,6.451613,84.112150,53.047935


개수 (pmshelter_count): 구별 대피소 수

총 수용인원 (pmshelter_total_capacity): 구별 전체 수용 가능 인원

가중 평균 점수

개수 점수 40%

총 수용인원 점수 60%
→ “많이 분포해 있으면서, 실제로 많은 인원을 수용할 수 있어야 안전하다”는 기준